In [379]:
import numpy as np
import pandas as pd
import time

In [380]:
dataset = pd.read_csv('ca-GrQc-new.csv')
nodes = np.concatenate((dataset['node_1'].unique(), dataset['node_2'].unique()))
nodes = np.unique(nodes)
e = 0
n = nodes.size

In [381]:
"""
for j in range(0, len(nodes)):
    dataset = dataset.replace(nodes[j], j)
"""

'\nfor j in range(0, len(nodes)):\n    dataset = dataset.replace(nodes[j], j)\n'

In [382]:
edges = dataset.iloc[:, :].values

In [383]:
G = {}

for i in nodes:
    #print(i)
    G[i] = []
    
for ed in edges:
    if ed[0] != ed[1]:
        G[ed[0]].append(ed[1])
        G[ed[1]].append(ed[0])
        e += 1

In [384]:
def densest(G, nodes, n, e):
    prev_core = None
    densities = []
    deg = {}
    bucket = []
    pos = {}
    vert = []
    edges = e
    vertices = n
    me = e
    mv = n
    posmc = None
    mcore = 0
    mden = 0
    md = 0
    
    for v in nodes:
        deg[v] = 0
        vert.append(None)
        pos[v] = None
    
    #easily parallelizable
    for v in nodes:
        d = 0
        for u in G[v]:
            d += 1
        
        deg[v] = d
        if d > md:
            md = d
    
    for d in range(md + 1):
        bucket.append(0)
    
    for v in nodes:
        bucket[deg[v]] += 1
        
    start = 0
    for d in range(md + 1):
        num = bucket[d]
        bucket[d] = start
        start += num
    
    #easily parallalizable
    for v in nodes:
        pos[v] = bucket[deg[v]]
        vert[pos[v]] = v
        bucket[deg[v]] += 1
    
    #easily parallalizable
    for d in range(md, 0, -1):
        bucket[d] = bucket[d - 1]
    bucket[0] = 0
    
    #print(bucket)
    
    deleted = 0
    temp = 0
    temp2 = 0
    for i in range(n):
        v = vert[i]
        core = deg[v]
        
        if prev_core != core:
            #print((edges - ((deleted + temp1 - temp2) / 2)), (n - i))
            #print(edges - ((deleted+temp)/2), (n - i))
            #print((edges - ((deleted + temp) / 2)))
            density = (edges - ((deleted+temp)/2)) / (n - i)
            densities.append([core, density])
            #deleted -= temp
            #temp1 -= temp2
            #temp2 = 0
            if density > mden:
                mden = density
                mcore = core
                me = edges
                mv = n - i
            prev_core = core
        
        #deleted += len(G[v])
        deleted += deg[v]
        
        for u in G[v]:
            #if pos[u] > pos[v]:
            #    edges -= 1
            if deg[u] <= core:
                deleted += 1
                temp -= 1
            else:
                temp += 1
            #elif deg[u] > core:
            #    temp += 1
                
            if deg[u] > deg[v]:
                du = deg[u]
                pu = pos[u]
                pw = bucket[du]
                w = vert[pw]
                
                if u != w:
                    pos[u] = pw
                    vert[pu] = w
                    pos[w] = pu
                    vert[pw] = u
                
                #deleted += 1
                
                #if deg[u] == deg[v] + 1:
                #    temp1 -= 1
                
                #if deg[u] > core:
                #    temp += 1
                
                bucket[du] += 1
                deg[u] -= 1
                
                
    
    #print(mden, me, mv)
    posmc = bucket[mcore]
    for i in range(posmc - 1, -1, -1):
        v = vert[i]
        
        if deg[v] < mden:
            break
            
        legits = 0
        for u in G[v]:
            if pos[u] >= posmc:
                legits += 1
        
        #print(mden, deg[v], legits)
        if legits > mden:
            w = vert[posmc - 1]
            pos[v] = posmc - 1
            vert[i] = w
            pos[w] = i
            vert[posmc - 1] = v
            posmc -= 1
            me += legits
            mv += 1
            mden = me / mv
            
    return densities, vert, pos, posmc, me, mv, mden

In [385]:
start_time = time.clock()
densities, vert, pos, posmc, me, mv, mden = densest(G, nodes, n, e)
print(time.clock() - start_time, "seconds")

0.04023300000000063 seconds


In [386]:
me, mv

(14484, 46)

In [387]:
densities

[[0, 2.7630675314765356],
 [1, 2.763594733829422],
 [2, 3.4145408163265305],
 [3, 4.3214695752009185],
 [4, 5.54384858044164],
 [5, 7.264994547437295],
 [6, 9.34330985915493],
 [7, 10.747252747252746],
 [8, 11.540740740740741],
 [9, 12.330555555555556],
 [10, 13.006230529595015],
 [11, 13.085443037974683],
 [12, 13.928057553956835],
 [13, 13.981684981684982],
 [14, 14.386100386100386],
 [15, 14.831967213114755],
 [17, 15.346491228070175],
 [19, 16.512953367875646],
 [20, 16.49214659685864],
 [21, 17.28235294117647],
 [23, 18.29054054054054],
 [25, 19.60483870967742],
 [31, 19.5609756097561],
 [33, 19.008695652173913],
 [34, 20.061728395061728],
 [42, 22.391304347826086],
 [43, 21.5]]

In [388]:
mden

22.391304347826086